In [1]:
import numpy as np
from sklearn.metrics import matthews_corrcoef, confusion_matrix

In [2]:
def norm_acc(y_true, y_pred):
    assert y_true.shape == y_pred.shape, "Inconsistent shapes between true labels <{0:s}> and predicted " \
                                         "labels <{1:s}>.".format(str(y_true.shape), str(y_pred.shape))

    cfmat = confusion_matrix(y_true, y_pred)

    if cfmat.size == 1:
        nacc = 1.0

    else:
        TN = cfmat[0, 0]
        FN = cfmat[1, 0]

        FP = cfmat[0, 1]
        TP = cfmat[1, 1]

        if (TN + FP) != 0:
            TNR = TN / (TN + FP)
        else:
            TNR = 0.0

        if (TP + FN) != 0:
            TPR = TP / (TP + FN)
        else:
            TPR = 0.0


        nacc = (TNR + TPR) / 2

    return nacc

In [3]:
gt = np.array([[1, 0, 1], [1, 1, 0], [1, 1, 1], 
               [0, 1, 1], [1, 0, 0], [1, 0, 1], 
               [1, 1, 1], [0, 1, 1], [0, 0, 0],
               [0, 0, 1], [0, 1, 1]], dtype=np.uint8)
pred = np.array([[1, 1, 1], [1, 1, 0], [1, 1, 1], 
                 [0, 0, 0], [0, 0, 0], [1, 0, 0], 
                 [1, 1, 1], [0, 0, 0], [0, 0, 0],
                 [1, 0, 0], [1, 1, 1]], dtype=np.uint8)

In [4]:
normaccs = []
mccs = []

for i in range(gt.shape[0]):
    normaccs.append(norm_acc(gt[i], pred[i]))
    mccs.append(matthews_corrcoef(gt[i], pred[i]))

print("     -- Per Sample --")
print("  True   --  Predicted ")
for i in range(gt.shape[0]):
    print(gt[i], " -- ", pred[i])

print("\nAvg. Norm. Acc: {0:0.3f}".format(np.mean(normaccs)))
print("Avg. MCC: {0:0.3f}".format(np.mean(mccs)))

     -- Per Sample --
  True   --  Predicted 
[1 0 1]  --  [1 1 1]
[1 1 0]  --  [1 1 0]
[1 1 1]  --  [1 1 1]
[0 1 1]  --  [0 0 0]
[1 0 0]  --  [0 0 0]
[1 0 1]  --  [1 0 0]
[1 1 1]  --  [1 1 1]
[0 1 1]  --  [0 0 0]
[0 0 0]  --  [0 0 0]
[0 0 1]  --  [1 0 0]
[0 1 1]  --  [1 1 1]

Avg. Norm. Acc: 0.682
Avg. MCC: 0.091


/home/alberto/miniconda3/envs/albpy/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [5]:
nacc = norm_acc(gt.reshape(-1), pred.reshape(-1))
mcc =  matthews_corrcoef(gt.reshape(-1), pred.reshape(-1))

print("-- All --")
print("True:      ", gt.reshape(-1))
print("Predicted: ", pred.reshape(-1))
print("\nNorm. Acc: {0:0.3f}".format(nacc))
print("MCC: {0:0.3f}".format(mcc))

-- All --
True:       [1 0 1 1 1 0 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 1 1]
Predicted:  [1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0 0 1 1 1]

Norm. Acc: 0.710
MCC: 0.410
